In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2020, 8, 1)
end_date = datetime(2020, 8, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

print DATE_GRANULARITY_MAPPINGLIST["monthly"]
print DATE_GRANULARITY_MAPPINGLIST["weekly"]
print DATE_GRANULARITY_MAPPINGLIST["daily"]

In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

app_basic_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from mu.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;"""

segment_by_age_gender_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ag.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

segment_by_product_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from au.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

app_cross_app = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ca.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

app_retention = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from rt.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

granularity_list = ["monthly", "weekly", "daily"]

def get_plproxy_data(sql_str):
    for granularity in granularity_list:
        for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
            result = query(PLPROXY_DSN, sql_str.format(granularity=granularity, date=date_str))
            count = result[0][0]
            print count
            
def get_unified_data():
    print 1




get_plproxy_data(app_basic_sql)    



In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
app_basic_sql = """
select distinct app_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct app_id
    from mu.app_daily_2001_143441_202001
    where 
        date ='{date}'
$proxy$) tpl (app_id bigint) ;

"""
# DATE_GRANULARITY_MAPPINGLIST["weekly"] = ["2020-06-30"]

for date_str in DATE_GRANULARITY_MAPPINGLIST["monthly"]:
    try:
        # app x domain, domain x domain, domain x app
        for db_name in [ "domain_x_domain"]:
            result = query(PLPROXY_DSN, sql_template.format(date=date_str, db_name=db_name))
            count = result[0][0]
            affinity_sum = result[0][1]
            up_sum = result[0][2]
        print "{},{},{:.20g},{:.20g}".format(date_str,count, up_sum, affinity_sum )
    except Exception, e:
        print "{},{}".format(date_str, "ERROR")